In [1]:
import data
import os, glob
import matplotlib.pyplot as plt
import cv2
import numpy as np

from networks import CNNFeatureLSTM

import keras
import tensorflow as tf

Using TensorFlow backend.


In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:
d = data.ClogData('/ssd_icybox2_1TB/jason/clog-loss-data/', size='micro')

In [4]:
d.crop_videos()

CROPPING ALL VIDEOS TO SIZE  (224, 224) ...
100%|██████████| 4798/4798 [00:00<00:00, 114267.13it/s]


In [ ]:
from tqdm import tqdm

for i in tqdm(range(len(d.test))):
    d.load(i, train=False, ret=False)

 12%|█▏        | 1663/14160 [02:56<15:54:54,  4.58s/it]

In [5]:
from cae import cae
input_shape = (224,224,3)
latent_dim = 100
cae_model = cae(input_shape, latent_dim)

In [6]:
#train_data = d.load_train()

In [7]:
#train_data = np.concatenate(train_data)

In [8]:
#cae_model.fit(train_data, epochs=50, callbacks=[keras.callbacks.BaseLogger(), 
#                                                keras.callbacks.EarlyStopping(patience=2,monitor='val_loss')],
#              validation_split=0.2)

In [9]:
#cae_model.save_weights(path=d.path)

In [10]:
cae_model.load_weights(path=d.path)

In [11]:
cnn = cae_model

In [12]:
cnn._keras_api_names = 'cae'
cnn._name = 'cae'

In [13]:
cnn.predict = lambda x: cnn.encode(x)

In [14]:
model = CNNFeatureLSTM(cnn, cnn_dim=latent_dim)

In [15]:
model.summary()

CNN Model: cae
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional (Bidirectional (None, 128)               84480     
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                8256      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 92,801
Trainable params: 92,801
Non-trainable params: 0
_________________________________________________________________


In [16]:
from tensorflow.keras.callbacks import EarlyStopping

In [17]:
model.fit(d, epochs=50, callbacks=[EarlyStopping(monitor='val_matthewcorr', 
                                                 patience=3,
                                                 mode='max',
                                                 restore_best_weights=True)])

LOADING CNN FEATURES...

TRAINING CLASSIFICATION MODEL...
Train on 1919 samples, validate on 480 samples
Epoch 1/50
1919/1919 [==============================] - 7s 4ms/sample - loss: 0.6351 - accuracy: 0.6748 - matthewcorr: nan - val_loss: 0.5847 - val_accuracy: 0.7063 - val_matthewcorr: nan
Epoch 2/50
1919/1919 [==============================] - 2s 1ms/sample - loss: 0.5431 - accuracy: 0.7353 - matthewcorr: nan - val_loss: 0.5723 - val_accuracy: 0.7208 - val_matthewcorr: nan
Epoch 3/50
1919/1919 [==============================] - 2s 1ms/sample - loss: 0.4700 - accuracy: 0.7895 - matthewcorr: nan - val_loss: 0.5877 - val_accuracy: 0.7042 - val_matthewcorr: 0.1863
Epoch 4/50
1919/1919 [==============================] - 2s 1ms/sample - loss: 0.3689 - accuracy: 0.8515 - matthewcorr: 0.6202 - val_loss: 0.6366 - val_accuracy: 0.6875 - val_matthewcorr: 0.1286
Epoch 5/50
1919/1919 [==============================] - 2s 1ms/sample - loss: 0.2431 - accuracy: 0.9156 - matthewcorr: 0.7904 - val_lo

In [18]:
predictions = model.predict(d, train=False)

  0%|          | 26/14160 [00:33<5:07:19,  1.30s/it] 


KeyboardInterrupt: 